In [7]:
from adiab_mitig import XZchain, pauli_field
import openfermion as of
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os

sys = XZchain(5, hx=0/5, hz=1/5, hzz=-5/5) 
dt = 0.07 # Trotter step size

magnetization_qop = pauli_field(sys.nspins, 'Z')
magnetization = of.get_sparse_operator(magnetization_qop)

# Simulation
generate data for Fig.3. 
Takes a few minutes.

In [ ]:
if os.path.exists('data/magnetization/T_list.npy'):
    print('Data already exists, it will be loaded from files.'
          'If you want to re-generate it, delete the data/magnetization folder or disable this check.')
    
    T_list = np.load('data/magnetization/T_list.npy')
    aa_expvals = np.load('data/magnetization/refl_expvals.npy')
    aev_t0_expvals = np.load('data/magnetization/aev_t0_expvals.npy')
    aev_t10_expvals = np.load('data/magnetization/aev_t10_expvals.npy')
    aev_tinf_expvals = np.load('data/magnetization/aev_tinf_expvals.npy')
else:
    os.makedirs('data/magnetization', exist_ok=True)

    dt = 0.07
    T_list = np.logspace(1, np.log10(2000), 40)

    aa_expvals = np.zeros_like(T_list, dtype=complex)
    aev_t0_expvals = np.zeros_like(T_list, dtype=complex)
    aev_t10_expvals = np.zeros_like(T_list, dtype=complex)
    aev_tinf_expvals = np.zeros_like(T_list, dtype=complex)


    for i, T in enumerate(tqdm(T_list)):
        nsteps = int(T / dt)
        
        magnetization = 

        refl_expval = sys.adiabatic_expval(T, nsteps, magnetization)
        aev_t0_expval = sys.aev_expval(T/2, nsteps, 0, magnetization)
        aev_t10_expval = sys.aev_expval(T/2, nsteps, 10, magnetization)
        aev_tinf_expval = sys.aev_expval(T/2, nsteps, None, magnetization)
        
        aa_expvals[i] = refl_expval 
        aev_t0_expvals[i] = aev_t0_expval
        aev_t10_expvals[i] = aev_t10_expval
        aev_tinf_expvals[i] = aev_tinf_expval

    np.save('data/magnetization/T_list.npy', T_list)
    np.save('data/magnetization/refl_expvals.npy', aa_expvals)
    np.save('data/magnetization/aev_t0_expvals.npy', aev_t0_expvals)
    np.save('data/magnetization/aev_t10_expvals.npy', aev_t10_expvals)
    np.save('data/magnetization/aev_tinf_expvals.npy', aev_tinf_expvals)

# Plotting

In [ ]:
# Plot settings
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['lines.markersize'] = 4
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['font.size'] = 10

mcolors = ["#00455E", "navy", "chocolate", "seagreen"]
fcolors = ["#CCDADF", "#CCCCE6", "#F6E1D2", "#D5E8DD"]

In [ ]:
plt.figure(figsize=(3.54, 2.36))

plt.plot(T_list, aa_expvals + 1, color=mcolors[1], label = 'QAA')
plt.plot(T_list, np.abs(aev_t0_expvals + 1), '1', color=mcolors[0], label='$T_d=0$')
plt.plot(T_list, np.abs(aev_t10_expvals + 1), '+',  color=mcolors[2], label=f'$T_d=10$')
plt.plot(T_list, np.abs(aev_tinf_expvals + 1), '.',  color=mcolors[3], label=r'$T_d \to \infty$')

lend = 23 # point where the exp line ends
plt.text(T_list[lend], np.exp(-T_list[lend]/7), r'$e^{-\beta T}$', va='bottom', ha='center')
plt.plot(T_list[:lend], np.exp(-T_list[:lend]/7), ':k')

lstr = 7 # point where the power law lines start
plt.text(T_list[lstr-1], 0.2*T_list[lstr-1]**(-2), r'$\propto T^{-2}$', va='center', ha='right')
plt.plot(T_list[lstr:], 0.2*T_list[lstr:]**(-2), ':k')

plt.text(T_list[lstr-1], 0.05*T_list[lstr-1]**(-4), r'$\propto T^{-4}$', va='center', ha='right')
plt.plot(T_list[lstr:], 0.05*T_list[lstr:]**(-4), ':k')

plt.yscale('log')
plt.xscale('log')
plt.xlabel('Total sweep time $T$')
plt.ylabel('Reflection estimator error')
plt.legend() # loc = 'center left', bbox_to_anchor = (1,0.5))
plt.plot()